In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_data_dir=r"C:\FILES\Binary_Bird_Classification\Train"
validation_data_dir=r"C:\FILES\Binary_Bird_Classification\Validation"
image_width=100
image_height=100
batch_size=150


In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(directory=train_data_dir,
                                              target_size = (image_width, image_height),
                                              classes=['Bald Eagle','Snowy Owl'],
                                              class_mode = 'binary',
                                              batch_size=batch_size)

validation_generator = datagen.flow_from_directory(directory=validation_data_dir,
                                              target_size = (image_width, image_height),
                                              classes=['Bald Eagle','Snowy Owl'],
                                              class_mode = 'binary',
                                              batch_size=batch_size)

In [ ]:
vgg_architecture=VGG16(input_shape=(image_width, image_height,3), weights="imagenet", include_top=False)

In [ ]:
for layers in vgg_architecture.layers:
  layers.trainable = False

In [ ]:
model=Sequential()
model.add(vgg_architecture)
model.add(Flatten())
model.add(Dense(128, activation='relu',))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), epochs = 5, 
                              validation_data=validation_generator, validation_steps=len(validation_generator)
                              , verbose = 1)

In [ ]:
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

In [ ]:
model.save('classification_model.h5')